In [1]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [2]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [4]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [5]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_C_013' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_C_013
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.0006174413730543459
test_rmse: 0.0006353453694414728
test_r2: 0.056711762688774495
feature number: 3
train_rmse: 0.0006168713247711479
test_rmse: 0.0006362426562282719
test_r2: 0.05396463122893637
feature number: 4
train_rmse: 0.0006150542890904689
test_rmse: 0.0006373734583264556
test_r2: 0.050891450240710956
feature number: 5
train_rmse: 0.0006136686643773623
test_rmse: 0.0006389228779838784
test_r2: 0.045988380828949
feature number: 6
train_rmse: 0.0006098727775124768
test_rmse: 0.0006382861840337131
test_r2: 0.04812972267367401
feature number: 7
train_rmse: 0.0006079085727526282
test_rmse: 0.0006391145899970497
test_r2: 0.0452736234906582
feature number: 8
train_rmse: 0.0006070010869203448
test_rmse: 0.0006420380713462082
test_r2: 0.03656566323337128
feature number: 9
train_rmse: 0.0006058205459838959
test_rmse: 0.0006452945772631895
test_r2: 0.026326500903156157
feature number: 10
train_rmse: 0.00060553301105632

train_rmse: 0.0005951618278445545
test_rmse: 0.0006423693004111562
test_r2: 0.032707851506835875
feature number: 14
train_rmse: 0.000589320522566397
test_rmse: 0.0006427026242216684
test_r2: 0.0304542793272224
feature number: 15
train_rmse: 0.0005845004197817809
test_rmse: 0.0006361528428142627
test_r2: 0.048986724191073215
feature number: 16
train_rmse: 0.0005841485248253646
test_rmse: 0.0006384698088059376
test_r2: 0.04230588582136835
feature number: 17
train_rmse: 0.0005836240616965579
test_rmse: 0.0006403198747713041
test_r2: 0.03690622819460786
feature number: 18
train_rmse: 0.0005830443185574188
test_rmse: 0.0006417008047499651
test_r2: 0.03286968455342502
feature number: 19
train_rmse: 0.0005820001547209299
test_rmse: 0.0006435037288923725
test_r2: 0.027013147868650923
feature number: 20
train_rmse: 0.000581757593956847
test_rmse: 0.0006444959236214003
test_r2: 0.02366483110171037
Test_rmse_min = 1.fea_num: 2  2.rmse: 0.0006353268215037666
Test_r2_max = 1. fea_num: 2  2.r2: 0.05

train_rmse: 0.0006098756890366623
test_rmse: 0.0006382602431493599
test_r2: 0.04820933637370338
feature number: 7
train_rmse: 0.0006078970731671033
test_rmse: 0.0006389680704998544
test_r2: 0.04569115671548381
feature number: 8
train_rmse: 0.0006069883179502165
test_rmse: 0.0006418958102614701
test_r2: 0.03697518652584173
feature number: 9
train_rmse: 0.0006058525411487504
test_rmse: 0.0006451524907407474
test_r2: 0.026745503355214285
feature number: 10
train_rmse: 0.0006055646974132944
test_rmse: 0.0006468779704652994
test_r2: 0.021205882631025207
feature number: 11
train_rmse: 0.0005959041363001099
test_rmse: 0.0006395497903757836
test_r2: 0.040942652643215706
feature number: 12
train_rmse: 0.000595825777203202
test_rmse: 0.0006398043195707367
test_r2: 0.04019505757966724
feature number: 13
train_rmse: 0.0005952269732608039
test_rmse: 0.0006422774551147075
test_r2: 0.032997120563980555
feature number: 14
train_rmse: 0.0005894194067012741
test_rmse: 0.0006426040008350315
test_r2: 0.03

# 可以修改第一層的神經元數量

In [8]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(2, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [9]:
fea_num=[2,3,4,6,15] # 要測試的feature數量 神經元2

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_0/Input_C_013.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.004140113480797238
[0.0005057951631982602, 0.0005375329054948663, 0.0006782721321014198, 0.0006598890421194664, 0.015598105328684797, 0.0006468204562631463, 0.020608888878042, 0.0007337320691674296, 0.0007756424752109459, 0.000656456357690045]


test_r2_ave: -147.2704904005792
[-5.459879690583591e-07, 0.06545443329095546, -0.03609874104433208, 0.01837389128958311, -653.4381562434075, -0.015235209618375212, -819.000777797732, -0.26762861005686456, -0.017158653466561757, -0.013676529059066356]


feature number: 3
test_rmse_ave: 0.0010657346820927933
[0.0006065801597174595, 0.0005399960991660344, 0.004713736393600882, 0.0006811139408184303, 0.000617654755772518, 0.000654030245070589, 0.0007306798753423003, 0.0006720495517769974, 0.0007756274548631738, 0.0006658783447995469]


test_r2_ave: -4.9686372538835935
[-0.43822652928191275, 0.056869876051430235, -49.04074676602169, -0.04578836647533491, -0.02616333823760142, -0.03799400007382148, -0.03076437752480